In [41]:
import numpy as np

np.set_printoptions(threshold=10000, suppress=True)
import pandas as pd
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

### Importer ce jeu de données avec la librairie pandas (c.f. read_csv)

In [42]:
data = pd.read_csv('PubMed-multi-label-dataset.csv', sep=',', encoding="utf-8")
data.head(3)

,Title,abstractText,meshMajor,pmid,meshid,meshroot,A,B,C,D,E,F,G,H,I,J,L,M,N,Z
0,Expression of p53 and coexistence of HPV in pr...,Fifty-four paraffin embedded tissue sections f...,"['DNA Probes, HPV', 'DNA, Viral', 'Female', 'H...",8549602,"[['D13.444.600.223.555', 'D27.505.259.750.600....","['Chemicals and Drugs [D]', 'Organisms [B]', '...",0,1,1,1,1,0,0,1,0,0,0,0,0,0
1,Vitamin D status in pregnant Indian women acro...,The present cross-sectional study was conducte...,"['Adult', 'Alkaline Phosphatase', 'Breast Feed...",21736816,"[['M01.060.116'], ['D08.811.277.352.650.035'],...","['Named Groups [M]', 'Chemicals and Drugs [D]'...",0,1,1,1,1,1,1,0,1,1,0,1,1,1
2,[Identification of a functionally important di...,The occurrence of individual amino acids and d...,"['Amino Acid Sequence', 'Analgesics, Opioid', ...",19060934,"[['G02.111.570.060', 'L01.453.245.667.060'], [...","['Phenomena and Processes [G]', 'Information S...",1,1,0,1,1,0,1,0,0,0,1,0,0,0


In [43]:
data = data.sample(frac=0.1, random_state=0)

### Analyser votre jeu de données, essentiellement la target. Chaque texte est labélisée par un
ensemble de labels parmi les 14 labels suivants : Anatomy [A], Organisms [B], Diseases [C], Chemicals and Drugs [D], Analytical, Diagnostic and Therapeutic Techniques, and Equipment [E], Psychiatry and Psychology [F],Phenomena and Processes [G], Disciplines and Occupations [H], Anthropology, Education, Sociology, and Social Phenomena [I], Technology, Industry, and Agriculture [J], Information Science [L], Named Groups [M], Health Care [N], Geographicals [Z]

### Modéliser le problème d’apprentissage supervisé sur ces données.

On décide de travailler uniquement sur l'abstractText

In [44]:
corpus = data['abstractText']

### Traiter vos données textuelles en supprimant les bruits dans les textes et en les normalisant. 
Vous pouvez vous inspirer par exemple du code par ici1 (en l’améliorant s’il le faut) si vous utiliserez la librairie NLTK. Vous pouvez aussi utiliser d’autres librairies (A vos recherches). N’oubliez pas que cette étape de pré-traitement (preprocessing) dépend de vos données et du problème traité. Quelques traitements à faire sont :
- Suppression des ponctuations comme . , ! $( ) * % @ o Suppression des URLs
- Suppression des Stop words
- Transformation de tout le texte en minuscule.
- Tokenisation de vos textes
- Racinisation (Stemming)
- Lemmatisation (lemmatization) o Etc.

In [45]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words


def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words


def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words


def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems


def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas


def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words


def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    return normalize(words)

In [46]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/floriangeillon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
from tqdm import tqdm

words_data = []
for sample in tqdm(corpus):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)

    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    words = normalize(words)

    words_data.append(words)

100%|██████████| 5000/5000 [01:14<00:00, 66.94it/s]


In [48]:
import pickle

with open('words_data.pkl', 'wb') as f:
    pickle.dump(words_data, f)

In [49]:
import pickle

with open('words_data.pkl', 'rb') as f:
    words_data = pickle.load(f)

In [50]:
print(words_data[0])

['members', 'threezincfinger', 'family', 'transcription', 'factors', 'play', 'important', 'role', 'determining', 'basal', 'transcription', 'cloned', 'mouse', 'bteb3', 'mbteb3', 'new', 'member', 'basic', 'transcription', 'element', 'binding', 'protein', 'bteb', 'family', 'expressed', 'wide', 'variety', 'tissues', 'mbteb3', 'activates', 'transcription', 'simian', 'virus', 'forty', 'early', 'promoter', '4fold', 'tissuespecific', 'sm22alpha', 'promoter', '100fold', 'suggesting', 'like', 'bteb1', 'sp1', 'mbteb3', 'basal', 'transcription', 'factor']


### Séparer les données en jeu de données d’apprentissage et jeu de données de test (50-50).

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(words_data, data[
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'Z']], test_size=0.5, random_state=0)

### Proposer une fonction run_models permettant de comparer plusieurs modèles d’apprentissage (en fonction de votre modélisation) sur ces données. 
Pour les approches multi-label, vous utiliserez l’approche EnsembleClassifierChain et MultiOutputCLassifier du package sklearn.multioutput. Ces classifieurs nécessite un classifieur de base (base_estimator) dont vous avez le libre choix d’utilisation. Votre évaluation se basera sur les deux mesures hamming_loss et zero_one_loss.

In [52]:
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss
from sklearn.metrics import zero_one_loss


def run_models(X_train, y_train, X_test, y_test, models, file_names=[]):
    for model in models:
        print(model)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print("Hamming loss: ", hamming_loss(y_test, y_pred))
        print("Zero-one loss: ", zero_one_loss(y_test, y_pred))
        print()
        if len(file_names) > 0:
            with open(file_names[models.index(model)], 'wb') as f:
                pickle.dump(model, f)

On choisit d'utiliser un classifieur de base RandomForestClassifier

In [53]:
from sklearn.ensemble import RandomForestClassifier

base_estimator = RandomForestClassifier()

models = [ClassifierChain(base_estimator=base_estimator),
          MultiOutputClassifier(estimator=base_estimator)]

### Proposer une première vectorisation de vos données textuelles par une représentation TF-IDF.

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

X_train_tfidf = [' '.join(words) for words in X_train]
X_test_tfidf = [' '.join(words) for words in X_test]

X_train_tfidf = vectorizer.fit_transform(X_train_tfidf)
X_test_tfidf = vectorizer.transform(X_test_tfidf)

### Exécuter ensuite votre fonction run_models sur vos données et interpréter les résultats obtenus.
Vous pouvez toujours appliquer votre fonction sur les données pré-traitées et les données non pré-
traitées afin d’analyser l’apport de la partie pré-traitement de données.

In [56]:
run_models(X_train_tfidf, y_train, X_test_tfidf, y_test, models,
           ['models/ClassifierChain_ap.pkl', 'models/MultiOutputClassifier_ap.pkl'])

ClassifierChain(base_estimator=RandomForestClassifier())
Hamming loss:  0.16377142857142857
Zero-one loss:  0.8996

MultiOutputClassifier(estimator=RandomForestClassifier())
Hamming loss:  0.16354285714285716
Zero-one loss:  0.9036


Sans prétraitement

In [57]:
from sklearn.model_selection import train_test_split

corpus = data['abstractText']
vectorizer_sp = TfidfVectorizer(stop_words=None)

X_train_sp, X_test_sp, y_train_sp, y_test_sp = train_test_split(corpus, data[
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'Z']], test_size=0.5, random_state=0)

X_train_tfidf_sp = vectorizer_sp.fit_transform(X_train_sp)
X_test_tfidf_sp = vectorizer_sp.transform(X_test_sp)

In [58]:
run_models(X_train_tfidf_sp, y_train_sp, X_test_tfidf_sp, y_test_sp, models,
           ['models/ClassifierChain_sp.pkl', 'models/MultiOutputClassifier_sp.pkl'])

ClassifierChain(base_estimator=RandomForestClassifier())
Hamming loss:  0.16925714285714286
Zero-one loss:  0.9088

MultiOutputClassifier(estimator=RandomForestClassifier())
Hamming loss:  0.16814285714285715
Zero-one loss:  0.904


Pas énormément de différence entre les deux approches (avec et sans prétraitement) peut être à cause de RandomForestClassifier

### Appliquer la méthode SVD de réduction de dimensions (TruncatedSVD) afin de construire des "concepts" liés aux documents et aux termes. Elle permettra entre autres de résoudre les problèmes de synonymie (plusieurs mots avec un seul sens) et de polysémie (un seul mot avec plusieurs sens). 
La fonction suivante vous aidera à comprendre les concepts en affichant leurs mots les plus pertinents.

In [59]:
def print_top_words(model, feature_names, n_top_words): 
    for topic_idx, topic in enumerate(model.components_):
        message = "Concept #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [60]:
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=100, random_state=0)
SVD.fit(X_train_tfidf)

TruncatedSVD(n_components=100, random_state=0)

In [61]:
feature_names = vectorizer.get_feature_names_out()
print_top_words(SVD, feature_names, 10)

Concept #0: patients thousand ninety seven thirty group eighty cells study treatment
Concept #1: cells cell expression protein activity gene human binding mice dna
Concept #2: thousand ninety ci health women eighty seven thirty risk million
Concept #3: thousand cells ninety ci cell eighty cancer expression seven thirty
Concept #4: group levels serum rats glucose plasma significantly blood insulin increased
Concept #5: health cells women risk cell levels cancer expression care group
Concept #6: gene expression patients genes protein levels sequence mutations dna proteins
Concept #7: group groups patients health cells control cell care pain expression
Concept #8: group expression gene left renal coronary ventricular groups genes imaging
Concept #9: cancer breast women risk carcinoma tumor method treatment factors chemotherapy
Concept #10: cells group infection antibodies disease children serum virus women cases
Concept #11: group treatment cancer binding case activity thousand renal pati

Même chose avec les données sans prétraitement

In [63]:
SVD_sp = TruncatedSVD(n_components=100, random_state=0)
SVD_sp.fit(X_train_tfidf_sp)

feature_names_sp = vectorizer_sp.get_feature_names_out()
print_top_words(SVD_sp, feature_names_sp, 10)

Concept #0: the of and in to with was were for patients
Concept #1: patients with were group was and had treatment who years
Concept #2: cells cell expression in il induced levels protein cancer receptor
Concept #3: group the was were mg rats kg groups glucose min
Concept #4: and health to ci 95 care women risk for research
Concept #5: il in group is rats glucose health insulin induced exercise
Concept #6: cancer cells the cell ci 95 group breast tumor lung
Concept #7: group in gene expression groups patients genes the were control
Concept #8: group binding of patients the cancer glucose protein ci insulin
Concept #9: il patients for alpha cancer treatment dose therapy binding group
Concept #10: of health il cells hiv care infection were cell anti
Concept #11: il the ifn expression 95 ci inflammatory mrna gene type
Concept #12: and cancer case tumor glucose breast levels of acid treatment
Concept #13: and health care protein binding cell patients beta cells the
Concept #14: expression 

In [64]:
X_train_tfidf_svd = SVD.transform(X_train_tfidf)
X_test_tfidf_svd = SVD.transform(X_test_tfidf)

X_train_tfidf_sp_svd = SVD_sp.transform(X_train_tfidf_sp)
X_test_tfidf_sp_svd = SVD_sp.transform(X_test_tfidf_sp)

In [65]:
run_models(X_train_tfidf_svd, y_train, X_test_tfidf_svd, y_test, models,
           ['models/ClassifierChain_ap_svd.pkl', 'models/MultiOutputClassifier_ap_svd.pkl'])

ClassifierChain(base_estimator=RandomForestClassifier())
Hamming loss:  0.1626
Zero-one loss:  0.8984

MultiOutputClassifier(estimator=RandomForestClassifier())
Hamming loss:  0.16251428571428572
Zero-one loss:  0.8976


In [66]:
run_models(X_train_tfidf_sp_svd, y_train_sp, X_test_tfidf_sp_svd, y_test_sp, models,
           ['models/ClassifierChain_sp_svd.pkl', 'models/MultiOutputClassifier_sp_svd.pkl'])

ClassifierChain(base_estimator=RandomForestClassifier())
Hamming loss:  0.16762857142857143
Zero-one loss:  0.9092

MultiOutputClassifier(estimator=RandomForestClassifier())
Hamming loss:  0.16857142857142857
Zero-one loss:  0.9144


In [ ]:
import gensim

## Apprentissage du Word2vec

In [ ]:
corpus = corpus.apply(lambda line: gensim.utils.simple_preprocess((line)))

In [ ]:
corpus.head(3)

In [ ]:
#Ici le code suivant a besoin de votre corpus de texte où chaque texte est tokenizé (une liste de tokens)


In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count()
cores

In [ ]:
model_size = 100
model = gensim.models.Word2Vec(corpus, vector_size=model_size, sg=0, window=5, min_count=2, workers=cores - 1)

In [ ]:
for i in range(100):
    model.train(corpus, total_examples=len(corpus), epochs=1)
    print(i, end=' ')

In [ ]:
model.save('../models/Word2vec_entraine.h5')

In [ ]:
model = gensim.models.Word2Vec.load('../models/Word2vec_entraine.h5')

In [ ]:
len(model.wv.index_to_key)

In [ ]:
model.wv['easy']

In [ ]:
model.wv.most_similar(positive='thank', topn=10)

In [ ]:
model.wv.doesnt_match(['food', 'drink', 'play'])

In [ ]:
model.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)